# Auto Orthophotos Downloader Demo

This demo shows how to use the new `AutoOrthophotoDownloader` that automatically detects which WMS services are needed for a given area and downloads orthophotos from all relevant services.

This is particularly useful when your area of interest spans multiple German federal states, each with their own WMS service.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from shapely.geometry import box

# Import the new auto downloader
from orthophotos_downloader.data_scraping.auto_downloader import AutoOrthophotoDownloader, auto_download_orthophotos

## 1. Create a Test Area That Spans Multiple States

For this demo, we'll create a test area that spans the border between Bavaria (BY) and Baden-Württemberg (BW).

In [ ]:
# Create a test area that spans Bavaria and Baden-Württemberg border
# These coordinates are in EPSG:25832 (UTM Zone 32N)
# Area around Ulm/Neu-Ulm which is on the border
test_area_bbox = box(570000, 5370000, 580000, 5380000)  # 10km x 10km area

# Create a GeoDataFrame
test_area = gpd.GeoDataFrame([1], geometry=[test_area_bbox], crs="EPSG:25832")

# Plot the test area
fig, ax = plt.subplots(figsize=(10, 10))
test_area.plot(ax=ax, color='red', alpha=0.7, edgecolor='black')
ax.set_title('Test Area Spanning Multiple States')
plt.show()

print(f"Test area bounds: {test_area.bounds}")

## 2. Detect Which States Intersect With Our Area

Let's use the auto downloader to detect which German federal states intersect with our test area.

In [ ]:
# Create an auto downloader instance
auto_downloader = AutoOrthophotoDownloader(grid_spacing=1000)  # 1km tiles

# Detect intersecting states
intersecting_states = auto_downloader.detect_intersecting_states(test_area.geometry)

print(f"Found {len(intersecting_states)} intersecting states:")
for state_name, state_code, intersection_geom in intersecting_states:
    print(f"  - {state_name} ({state_code})")
    print(f"    Intersection area: {intersection_geom.area/1000000:.2f} km²")

## 3. Visualize the State Boundaries and Intersections

Let's visualize how our test area intersects with the German federal states.

In [ ]:
# Load German states for visualization
states_gdf = auto_downloader._load_german_states()

# Get the states that intersect with our area
intersecting_state_names = [s[0] for s in intersecting_states]
intersecting_states_gdf = states_gdf[states_gdf['name'].isin(intersecting_state_names)]

# Create visualization
fig, ax = plt.subplots(figsize=(12, 10))

# Plot all German states in light gray
states_gdf.plot(ax=ax, color='lightgray', alpha=0.5, edgecolor='white')

# Plot intersecting states in blue
intersecting_states_gdf.plot(ax=ax, color='lightblue', alpha=0.7, edgecolor='blue')

# Plot our test area in red
test_area.plot(ax=ax, color='red', alpha=0.8, edgecolor='darkred', linewidth=2)

# Add state name labels
for idx, row in intersecting_states_gdf.iterrows():
    centroid = row.geometry.centroid
    ax.annotate(row['name'], xy=(centroid.x, centroid.y), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=10, ha='left', va='bottom',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

ax.set_title('Test Area and Intersecting German States', fontsize=14)
ax.legend(['Other states', 'Intersecting states', 'Test area'])
plt.tight_layout()
plt.show()

## 4. Download Orthophotos Automatically

Now let's use the auto downloader to download RGB orthophotos from all relevant WMS services.

In [ ]:
# Set up download parameters
AREA_NAME = "multi_state_test_area"
OUT_PATH = Path("./auto_download_results")
GRID_SPACING = 2000  # 2km tiles (larger for demo to reduce download time)

# Create output directory
OUT_PATH.mkdir(exist_ok=True)

print(f"Starting automatic download for area: {AREA_NAME}")
print(f"Output path: {OUT_PATH}")
print(f"Grid spacing: {GRID_SPACING}m")

In [ ]:
# Method 1: Using the convenience function
results = auto_download_orthophotos(
    area_name=AREA_NAME,
    area_polygon=test_area.geometry,
    out_path=OUT_PATH,
    grid_spacing=GRID_SPACING,
    image_type="RGB",
    filename_prefix="auto_rgb"
)

print(f"\nDownload completed! Results:")
total_images = 0
for state_name, area_dataset in results.items():
    num_images = len(area_dataset.images)
    total_images += num_images
    print(f"  {state_name}: {num_images} images")

print(f"\nTotal images downloaded: {total_images}")

## 5. Alternative: Using the Class-Based Approach

You can also use the `AutoOrthophotoDownloader` class directly for more control.

In [ ]:
# Method 2: Using the class directly
auto_downloader = AutoOrthophotoDownloader(grid_spacing=GRID_SPACING)

# Download RGB images
rgb_results = auto_downloader.download_rgb_images_auto(
    area_name=f"{AREA_NAME}_class_method",
    area_polygon=test_area.geometry,
    out_path=OUT_PATH / "class_method",
    filename_prefix="class_rgb"
)

print(f"\nClass-based download completed! Results:")
total_images = 0
for state_name, area_dataset in rgb_results.items():
    num_images = len(area_dataset.images)
    total_images += num_images
    print(f"  {state_name}: {num_images} images")

print(f"\nTotal images downloaded: {total_images}")

## 6. Download CIR Images (Color Infrared)

If CIR images are available for the intersecting states, you can also download them automatically.

In [ ]:
# Download CIR images
try:
    cir_results = auto_downloader.download_cir_images_auto(
        area_name=f"{AREA_NAME}_cir",
        area_polygon=test_area.geometry,
        out_path=OUT_PATH / "cir",
        filename_prefix="auto_cir"
    )
    
    print(f"\nCIR download completed! Results:")
    total_images = 0
    for state_name, area_dataset in cir_results.items():
        num_images = len(area_dataset.images)
        total_images += num_images
        print(f"  {state_name}: {num_images} CIR images")
    
    print(f"\nTotal CIR images downloaded: {total_images}")
    
except Exception as e:
    print(f"CIR download failed: {e}")
    print("This may be because CIR images are not available for all intersecting states.")

## 7. Download RGBI Images (RGB + Infrared)

If both RGB and CIR images are available, you can download and merge them into RGBI images.

In [ ]:
# Download RGBI images (RGB + Infrared merged)
try:
    rgbi_results = auto_downloader.download_rgbi_images_auto(
        area_name=f"{AREA_NAME}_rgbi",
        area_polygon=test_area.geometry,
        out_path=OUT_PATH / "rgbi"
    )
    
    print(f"\nRGBI download completed! Results:")
    total_images = 0
    for state_name, area_dataset in rgbi_results.items():
        num_images = len(area_dataset.images)
        total_images += num_images
        print(f"  {state_name}: {num_images} RGBI images")
    
    print(f"\nTotal RGBI images downloaded: {total_images}")
    
except Exception as e:
    print(f"RGBI download failed: {e}")
    print("This may be because CIR images are not available for all intersecting states.")

## 8. Inspect the Downloaded Results

Let's examine what was downloaded and the file structure.

In [ ]:
# List all downloaded files
print("Downloaded files structure:")
for root, dirs, files in OUT_PATH.walk():
    level = root.relative_to(OUT_PATH).parts.__len__()
    indent = ' ' * 2 * level
    print(f"{indent}{root.name}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

In [ ]:
# Show summary statistics
if results:
    print("\nDownload Summary:")
    print("=" * 50)
    
    for state_name, area_dataset in results.items():
        print(f"\n{state_name}:")
        print(f"  Number of images: {len(area_dataset.images)}")
        print(f"  Output path: {area_dataset.out_path}")
        print(f"  Buffer size: {area_dataset.buffer_size}m")
        
        if area_dataset.images:
            # Show details of first image
            first_image = area_dataset.images[0]
            print(f"  First image details:")
            print(f"    Resolution: {first_image.resolution_m}m/pixel")
            print(f"    Size: {first_image.width_px}x{first_image.height_px} pixels")
            print(f"    CRS: {first_image.crs}")
            print(f"    Download time: {first_image.download_time:.2f}s")

## Summary

The `AutoOrthophotoDownloader` provides a convenient way to download orthophotos from areas that span multiple German federal states. It:

1. **Automatically detects** which German federal states intersect with your area of interest
2. **Finds the intersection geometry** for each state
3. **Instantiates the appropriate WMS downloader** for each state
4. **Downloads orthophotos** from each relevant WMS service
5. **Organizes results** by state for easy access

### Key Features:
- **Automatic detection** of required WMS services
- **Support for RGB, CIR, and RGBI** image types
- **Handles state boundaries** and intersections automatically
- **Organized output** with separate directories for each state
- **Error handling** for states where downloads fail
- **Convenience function** for simple use cases
- **Class-based approach** for more control

This eliminates the need to manually determine which WMS services cover your area and handle the complexity of downloading from multiple sources.